In [1]:
import pandas as pd
import sql_functions as sql_int

In [2]:
tournament_result = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/results_esl_summer_2022.csv')

In [3]:
df_tournament_result = tournament_result.copy()
tournament_result

,playerName,round,group,points,tournament
0,Momor vig,1,A,2,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
1,gluteus,1,A,3,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
2,5454XD,1,A,8,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
3,kevinparkertft,1,A,1,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
4,TFTMarx,1,A,6,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
...,...,...,...,...,...
315,5454XD,5,H,7,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
316,Leinad TFT,5,H,4,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
317,Momor vig,5,H,1,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...
318,Memoooo,5,H,3,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...


#### changing points into placement

### Testing names are real

In [4]:
def check_new(df_new,df_exist,columns):
    df_new = df_new.copy()
    df_exist = df_exist.copy()
    df_exist['already_exist'] = 'Yes'
    drop_columns = df_exist.columns.difference(df_new.columns)
    df_concat = pd.concat([df_exist,df_new],ignore_index=True)
    df_concat = (df_concat
                 .drop_duplicates(subset=[columns],keep=False)
                 .query('already_exist != "Yes"')
                 .drop(columns=drop_columns)
    )
    return df_concat

In [5]:
df_player = tournament_result[['playerName']].copy().drop_duplicates()

schema = 'tft_table'
table = 'alternative_name'
sql_query = f'SELECT player_id,player_name FROM {schema}.{table};'
alternative_name = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['player_id','player_name'])
df_alternative_name = alternative_name.copy()
df_player = df_player.drop_duplicates().rename(columns={'playerName': 'import_name' })
df_player_import = check_new(df_player,
                             df_alternative_name.rename(columns={'player_name' :'import_name' }),
                             'import_name')
df_player_import

,import_name


### aDDING STUFF

In [6]:
df_tournament_result = df_tournament_result.dropna()
if 'placement' in df_tournament_result.columns: 
    pass 
else: 
    df_tournament_result['placement'] = df_tournament_result['points'].apply(lambda x: 9-int(x))
    df_tournament_result = df_tournament_result.drop(columns='points')
           
df_tournament_result[df_tournament_result.duplicated(keep= False, subset=['tournament','group','round','placement']) == True].sort_values(by=['tournament','round','group'])

,playerName,round,group,tournament,placement


In [7]:
df_tournament_result = df_tournament_result.drop_duplicates()

#### create Result tables

In [8]:
df_tournament_result = df_tournament_result.drop_duplicates()

df_lobby_result = (pd.pivot_table(df_tournament_result,values='playerName',columns='placement',index=['tournament','round','group'],aggfunc='first')
                   .reset_index()
                   .rename(columns={1:'winner',2:'runnerup',3:'third',4:'fourth',5:'fifth',6:'sixth',7:'seventh',8:'eighth'})
                   .drop(columns=[9],errors='ignore')
                   )
df_lobby_result

placement,tournament,round,group,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth
0,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,A,5454XD,JohnSchmieri,TFTMarx,Artilleryy,jonobo,gluteus,Momor vig,kevinparkertft
1,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,B,salvyyy,TwoBiers,SquG3,leduck_lol,Leinad TFT,Giovanni1996,phinphinthao,Memoooo
2,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,C,JohnSchmieri,kevinparkertft,Memoooo,phinphinthao,Momor vig,gluteus,SquG3,salvyyy
3,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,D,leduck_lol,Giovanni1996,TwoBiers,Artilleryy,jonobo,Leinad TFT,5454XD,TFTMarx
4,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,E,leduck_lol,gluteus,Leinad TFT,salvyyy,Giovanni1996,Momor vig,TwoBiers,Memoooo
5,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,F,kevinparkertft,SquG3,TFTMarx,Artilleryy,JohnSchmieri,5454XD,phinphinthao,jonobo
6,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,G,jonobo,TFTMarx,phinphinthao,Giovanni1996,leduck_lol,JohnSchmieri,SquG3,kevinparkertft
7,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,1,H,Leinad TFT,TwoBiers,gluteus,salvyyy,5454XD,Artilleryy,Memoooo,Momor vig
8,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,2,A,gluteus,5454XD,JohnSchmieri,Momor vig,TFTMarx,jonobo,Artilleryy,kevinparkertft
9,ESL MEISTERSCHAFT BY SUMMONER'S INN - SOMMER 2...,2,B,salvyyy,Memoooo,TwoBiers,SquG3,phinphinthao,Leinad TFT,leduck_lol,Giovanni1996


#### Take tournament ID's

In [9]:
schema = 'tft_table'
table = 'tournament'
sql_query = f'''select name, tournament_id
                FROM {schema}.{table};
                '''

df_tounament_id = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['tournament','tournament_id'])

df_tounament_push = (pd.merge(df_lobby_result,df_tounament_id, how='left',left_on='tournament',right_on='tournament')
                     .drop(columns='tournament')
                     )

#### Checking tournament result already exists

In [10]:
table = 'lobby'
sql_query = f'''SELECT tournament_id,round,party
                FROM {schema}.{table};
                '''
df_existing_lobby = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                    columns=['tournament_id','round','group'])
df_existing_lobby['existing'] = 'Yes'
df_no_dups = pd.concat([df_tounament_push,df_existing_lobby],ignore_index=True)
df_no_dups = df_no_dups.drop_duplicates(subset=['tournament_id','group','round'],keep=False).rename(columns={'group': 'party'})

df_no_dups = (df_no_dups
                  .drop_duplicates(subset=['tournament_id','round','party'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['existing'],axis=1)
                  )

In [11]:
df_no_dups

,round,party,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth,tournament_id
0,1,A,5454XD,JohnSchmieri,TFTMarx,Artilleryy,jonobo,gluteus,Momor vig,kevinparkertft,27
1,1,B,salvyyy,TwoBiers,SquG3,leduck_lol,Leinad TFT,Giovanni1996,phinphinthao,Memoooo,27
2,1,C,JohnSchmieri,kevinparkertft,Memoooo,phinphinthao,Momor vig,gluteus,SquG3,salvyyy,27
3,1,D,leduck_lol,Giovanni1996,TwoBiers,Artilleryy,jonobo,Leinad TFT,5454XD,TFTMarx,27
4,1,E,leduck_lol,gluteus,Leinad TFT,salvyyy,Giovanni1996,Momor vig,TwoBiers,Memoooo,27
5,1,F,kevinparkertft,SquG3,TFTMarx,Artilleryy,JohnSchmieri,5454XD,phinphinthao,jonobo,27
6,1,G,jonobo,TFTMarx,phinphinthao,Giovanni1996,leduck_lol,JohnSchmieri,SquG3,kevinparkertft,27
7,1,H,Leinad TFT,TwoBiers,gluteus,salvyyy,5454XD,Artilleryy,Memoooo,Momor vig,27
8,2,A,gluteus,5454XD,JohnSchmieri,Momor vig,TFTMarx,jonobo,Artilleryy,kevinparkertft,27
9,2,B,salvyyy,Memoooo,TwoBiers,SquG3,phinphinthao,Leinad TFT,leduck_lol,Giovanni1996,27


In [12]:
sql_int.push_to_database(df_no_dups,table,sql_int.get_engine_alchemy(),schema)

The lobby table was imported successfully.
